In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

In [ ]:
parcel_update_sql = '''   
SELECT [parcelid_2015]
      ,jurisdiction_id
      ,[du_2015]
      ,u.du_2017 as update_du_2017
      ,p.du_2017 as parcel_du_2017
      ,[du_urb]
      ,[cap_urb]
      ,p.capacity
      ,[buildout]
      ,[cap_remaining_old]
      ,u.cap_remaining_new as update_cap_remaining_new
      ,p.cap_remaining_new as parcel_cap_remaining_new
  FROM  [urbansim].[urbansim].[parcel_update_2017] u
  JOIN [urbansim].[urbansim].[parcel] p on p.parcel_id = u.parcelid_2015
  where p.capacity > 0 OR cap_urb > 0'''
parcel_update_df = pd.read_sql(parcel_update_sql, mssql_engine)
parcel_update_df.fillna(0,inplace=True)

## Check data

In [ ]:
len(parcel_update_df)

In [ ]:
parcel_update_df.loc[parcel_update_df.update_cap_remaining_new.isnull()]

In [ ]:
parcel_update_df.loc[parcel_update_df.capacity.isnull()]

In [ ]:
parcel_update_df.loc[parcel_update_df.update_cap_remaining_new == 0].head()

In [ ]:
parcel_update_df.loc[parcel_update_df.capacity==0].head()

## add jurisdiction names

In [ ]:
jurisdictions_names_sql = '''
    SELECT zone, name
      FROM data_cafe.ref.geography_zone
     WHERE geography_type_id = 150
'''
jur_name_df = pd.read_sql(jurisdictions_names_sql, mssql_engine)

## plot data

In [ ]:
from matplotlib.patches import Rectangle
fig, axes = plt.subplots(nrows=10, ncols=2)
for j, jur in enumerate(jur_name_df.zone.unique().tolist()):
    df_plot = parcel_update_df.loc[parcel_update_df.jurisdiction_id == jur].copy()
    df_plot = pd.merge(df_plot,jur_name_df[['zone','name']],left_on='jurisdiction_id',right_on='zone')
    # df_plot.plot(kind='scatter',x='cap_urb', y='capacity',ax=axes.flat[j],figsize=(10,30))
    # df_plot.plot(kind='scatter',x='update_cap_remaining_new', y='capacity',ax=axes.flat[j],figsize=(10,30))
    # df_plot.plot(kind='scatter',x='update_du_2017', y='parcel_du_2017',ax=axes.flat[j],figsize=(10,30))
    df_plot.plot(kind='scatter',x='update_cap_remaining_new', y='parcel_cap_remaining_new',ax=axes.flat[j],figsize=(10,30))
    axes.flat[j].set_xlabel("parcel_update_2017 cap_urb",size=12)
    axes.flat[j].set_ylabel("urbansim.parcel capacity",size=12)
    axes.flat[j].set_title('QC of parcel_update_2017',size=14) 
    extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
    axes.flat[j].legend([extra], [df_plot.name[0]],fontsize=14)
    fig.tight_layout()
fig.savefig('qc_parcel_update_2017_capacity_vs_cap_remaining_new.png', format='png', dpi=300)